# Creating your own pdf

A fundamental design choice of zfit is the ability to create custom pdfs and functions in an easy way.

There are two ways, an easier for most use-cases and an advanced way of creating your own pdf.

## The simple way

While the same works for functions, an example with a PDF is shown here